# 1. Selenium이란?
* 웹브라우저를 자동으로 제어하는 라이브러리
* 원래 다양한 웹브라우저를 자동으로 테스트하는 테스트 도구
* 동적 웹사이트에서 정보를 가져오는데 활용

In [15]:
# !pip install selenium
# !pip install webdriver-manager

In [16]:
import selenium
print(selenium.__version__)

4.31.1


In [17]:
import platform
platform.architecture()

('64bit', 'ELF')

In [18]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-logging'])
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36")
options.add_argument('lang=ko_KR')

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) #웹브라우저 해상도 조절
driver.get("https://search.shopping.naver.com/book/home")

In [20]:
#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > input
# _lnbSearch_search_input_YVyzR N=a:sech.input
serch_box = driver.find_element(By.CSS_SELECTOR, "#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > input")
serch_box.send_keys("핀테크") #+Keys.ENTER)

In [23]:
#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > button
search_button = driver.find_element(By.CSS_SELECTOR, "#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > button")
search_button.click()

# 마우스를 스크롤 해서 40개 정보 가져오기
* window.scrollTo({시작위치}, {step})

In [24]:
# final_result = pd.DataFrame()
result = {}
for page in range(1,7):
    # 마우스 스크롤로 40개 정보 가져옴
    y = 0
    y_step = 1000
    for scroll_times in range(1, 8):
        y = y + y_step
        driver.execute_script(f"window.scrollTo({0}, {y})")
        time.sleep(2)
    page_html = driver.page_source
    soup = bs(page_html, 'lxml')
    book_list = soup.select(".list_book > li")
    
    for book in book_list:
        title = book.select_one("span.bookListItem_text__SL9m9").text # 책 제목
        # 저자와 출판사
        if len(book.select("span.bookListItem_define_data__IUMgt")) != 0:
            if book.select_one(".bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt") != None:
                author = book.select_one(".bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt").text  # 저자
            else:
                author = ""
            if book.select_one(".bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt") != None:   
                publisher = book.select_one(".bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt").text # 출판사
            else:
                publisher = ""
        else:
            author = ""
            publisher = ""
        # 출간일
        if book.select_one("div.bookListItem_detail_date__s7KQe") != None:    
            pub_date = book.select_one("div.bookListItem_detail_date__s7KQe").text.strip(". ").replace(".","-") # 출간일
        else:
            pub_date = ""

        # 가격부분
        if book.select("span.bookPrice_price__OagxI > em") != None:
            print("가격부분:",len(book.select("span.bookPrice_price__OagxI > em")))
            if len(book.select("span.bookPrice_price__OagxI > em")) == 2:
                price = book.select("span.bookPrice_price__OagxI > em")[0].text.replace(",","") # 종이책 가격
                eb_price = book.select("span.bookPrice_price__OagxI > em")[1].text.replace(",","") # e북 가격
            elif len(book.select("span.bookPrice_price__OagxI > em")) == 1:
                price = book.select("span.bookPrice_price__OagxI > em")[0].text.replace(",","") # 종이책 가격
                eb_price = 0
        else:
            price = 0
            eb_price = 0
        keys =('title', 'author', 'publisher', 'pub_date', 'price', 'eb_price')
        data = (title, author, publisher, pub_date, price, eb_price)
        for key, item in zip(keys, data):
            result.setdefault(key, []).append(item)
    # result_df = pd.DataFrame(result)
    # final_result = pd.concat([final_result, result_df])
    # 다음 페이지 이동 버튼 누르기
    npbutton = "#container > div > div.bookSearch_book_search__Fm5Em > div > div.Paginator_list_paging__hUKRc > button.Paginator_btn_next__7NiBL"
    next_page_button = driver.find_element(By.CSS_SELECTOR, npbutton)
    next_page_button.click()
driver.close()

가격부분: 2
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 1
가격부분: 0
가격부분: 1
가격부분: 0
가격부분: 0
가격부분: 0
가격부분: 0
가격부분: 1
가격부분: 1
가격부분: 0
가격부분: 1
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 2
가격부분: 2
가격부분: 1
가격부분: 0
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 1
가격부분: 0
가격부분: 2
가격부분: 1
가격부분: 0
가격부분: 2
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 0
가격부분: 0
가격부분: 0
가격부분: 1
가격부분: 0
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 0
가격부분: 2
가격부분: 0
가격부분: 2
가격부분: 1
가격부분: 1
가격부분: 2
가격부분: 0
가격부분: 1
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 2
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1
가격부분: 1


In [ ]:
# final_result

In [25]:
import pandas as pd
book_result= pd.DataFrame(result)
book_result

,title,author,publisher,pub_date,price,eb_price
0,핀테크,한석주,커뮤니케이션북스,2015-11-01,11400,8640
1,핀테크 (IT와 금융이 만나는 새로운 세상),강창호,한빛미디어,2015-06-05,12600,0
2,핀테크 전쟁 (세계 금융 시장의 거대한 트렌드 | 새로운 돈의 시대가 온다),브렛 킹,예문,2015-06-12,13500,9450
3,핀테크 4.0 (핀테크 혁명과 금융의 미래),김종현,한국금융연수원,2022-05-24,17100,0
4,핀테크 혁명 (모바일 은행 설립자가 알려주는 핀테크 시대의 돈 관리 기술),앤 보덴,유엑스리뷰,2020-03-03,20700,0
...,...,...,...,...,...,...
235,교보문고 교보문고 미래유망산업분야 기술용어사전 - 인공지능 바이오 핀테크 빅데이터,,,,59400,0
236,[보리보리/아이와함께]NEW Live 과학 첨단과학 15 - 핀테크와 가상화폐 (천...,,,,11700,0
237,교보문고 교보문고 핀테크 산업분야 취업가이드 - 네이버 카카오뱅크 카카오페이 KG이니,,,,28800,0
238,"핀테크에반젤리스트 - 디지털 금융, 핀테크 스타트업, 금융 혁신, 핀테크 생태계, ...",,씨익북스,2024-05-11,3100,0


In [132]:
# 다음 페이지 이동 버튼 누르기
npbutton = "#container > div > div.bookSearch_book_search__Fm5Em > div > div.Paginator_list_paging__hUKRc > button.Paginator_btn_next__7NiBL"
next_page_button = driver.find_element(By.CSS_SELECTOR, npbutton)
next_page_button.click()

In [134]:
# 마우스 스크롤로 40개 정보 가져옴
y = 0
y_step = 1000
for scroll_times in range(1, 8):
    y = y + y_step
    driver.execute_script(f"window.scrollTo({0}, {y})")
    time.sleep(2)
page_html = driver.page_source
soup = bs(page_html, 'lxml')
book_list = soup.select(".list_book > li")
for book in book_list:
    title = book.select_one("span.bookListItem_text__SL9m9").text # 책 제목
    
    if len(book.select("span.bookListItem_define_data__IUMgt")) != 0:
        author = book.select_one(".bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt").text if book.select_one(".bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt") != None else ""# 저자
        publisher = book.select_one(".bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt").text if book.select_one(".bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt") != None else "" # 출판사
    else:
        author = ""
        publisher = ""
        
    if book.select_one("div.bookListItem_detail_date__s7KQe") != None:    
        pub_date = book.select_one("div.bookListItem_detail_date__s7KQe").text.strip(". ").replace(".","-") # 출간일
    else:
        pub_date = ""
    
    
    if book.select("span.bookPrice_price__OagxI > em") != None:
        print("가격부분:",len(book.select("span.bookPrice_price__OagxI > em")))
        if len(book.select("span.bookPrice_price__OagxI > em")) == 2:
            price = book.select("span.bookPrice_price__OagxI > em")[0].text.replace(",","") # 종이책 가격
            eb_price = book.select("span.bookPrice_price__OagxI > em")[1].text.replace(",","") # e북 가격
        elif len(book.select("span.bookPrice_price__OagxI > em")) == 1:
            price = book.select("span.bookPrice_price__OagxI > em")[0].text.replace(",","") # 종이책 가격
            eb_price = 0
    else:
        price = 0
        eb_price = 0
    
    print(title, author, publisher, pub_date, price, eb_price)

가격부분: 1
[온누리북스] NEW Live 과학 첨단과학 15 : 핀테크와 가상화폐    11700 0
가격부분: 1
기업 클라우드 IT 전략 빅데이터, 사물인터넷, 핀테크의 인프라 전략    16200 0
가격부분: 1
핀테크와 블록체인의 미래 - 핀테크는 어떻게 디지털 혁명을 가져왔나    31500 0
가격부분: 1
핀테크와 블록체인의 미래 - 핀테크는 어떻게 디지털 혁명을 가져왔나    31500 0
가격부분: 1
핀테크와 블록체인의 미래   핀테크는 어떻게 디지털 혁명을 가져왔나    31500 0
가격부분: 1
기업 클라우드 IT 전략 빅데이터, 사물인터넷, 핀테크의 인프라 전략    16200 0
가격부분: 1
[보리보리/웅진북센]핀테크와 GDPR(1)핀테크를위한개인정보보호    17100 0
가격부분: 1
교보문고 교보문고 부자  관상  기술 - 부자들은 알고 있는 핀테크 시대의 행    14400 0
가격부분: 1
[보리보리/유니오니아시아]웹 3.0 시대 핀테크로 부자되기 / 보민출판사    14400 0
가격부분: 1
[보리보리/이노플리아]Live 과학 첨단과학 15 핀테크와 가상화폐    11700 0
가격부분: 1
[보리보리/웅진북센]핀테크(기회를잡아라)돈의흐름을바꾸는금융대혁명    12600 0
가격부분: 1
한 발 앞서 정복하는 핀테크 이야기 일상의 변화에서 미래의 경제 혁명까지    13500 0
가격부분: 1
[보리보리/밀크북]NEW Live 과학 첨단과학 15 핀테크와 가상화폐    11700 0
가격부분: 1
[무.배] 핀테크와 블록체인의 미래 - 핀테크는 어떻게 디지털 혁명을 가져왔나    31500 0
가격부분: 1
카카오와 네이버는 어떻게 은행이 되었나 : 핀테크 트렌드로 보는 밀레니얼    14400 0
가격부분: 1
블록체인 핀테크 기술 정책동향과 정보보호 분야별 산업 실태분석 양장본    342000 0
가격부분: 1
[보리보리/이노플리아]핀테크와 GDPR(2)개인 데이터 주권을 위한 접근법    17100 0
가격

In [124]:
title

'[eBook] 내일의 핀테크 (체험판)'

In [125]:
book.select_one(".bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt").text

AttributeError: 'NoneType' object has no attribute 'text'

In [126]:
book.select_one(".bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt").text

'매일경제신문사'

In [121]:
book.select_one("div.bookListItem_detail_date__s7KQe")

In [118]:
len(book.select("span.bookListItem_define_data__IUMgt"))

0

In [108]:
book.select_one("span.bookListItem_text__SL9m9").text

'내일의 핀테크'

In [102]:
book.select("span.bookPrice_price__OagxI > em")[0].text

'27,000'

In [93]:
book.select("span.bookPrice_price__OagxI")[0].text

'2,790원무료'

In [98]:
book.select("span.bookPrice_price__OagxI")[0].text

'최저 9,500원'

In [42]:
book.select("span.bookPrice_price__OagxI")[0].text

'2,790원무료'

In [ ]:
#container > div > div.bookSearch_book_search__Fm5Em > div > 

In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time

# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-logging'])
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36")
options.add_argument('lang=ko_KR')

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) #웹브라우저 해상도 조절
driver.get("https://search.shopping.naver.com/book/home")

In [44]:
#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > input
# _lnbSearch_search_input_YVyzR N=a:sech.input
serch_box = driver.find_element(By.CSS_SELECTOR, "#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > input")
serch_box.send_keys("핀테크") #+Keys.ENTER)

In [45]:
#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > button
search_button = driver.find_element(By.CSS_SELECTOR, "#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > button")
search_button.click()

In [66]:
# 다음 페이지 이동 버튼 누르기
npbutton = "#container > div > div.bookSearch_book_search__Fm5Em > div > div.Paginator_list_paging__hUKRc > button.Paginator_btn_next__7NiBL"
next_page_button = driver.find_element(By.CSS_SELECTOR, npbutton)
next_page_button.click()

In [105]:
# 마우스 스크롤로 40개 정보 가져옴
y = 0
y_step = 1000
for scroll_times in range(1, 8):
    y = y + y_step
    driver.execute_script(f"window.scrollTo({0}, {y})")
    time.sleep(2)
page_html = driver.page_source
soup = bs(page_html, 'lxml')
book_list = soup.select(".list_book > li")
for book in book_list:
    title = book.select_one("span.bookListItem_text__SL9m9").text # 책 제목
    if len(book.select("span.bookListItem_define_data__IUMgt")) != 0:
        if len(book.select("span.bookListItem_define_data__IUMgt")) == 2:
            author = book.select_one("bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt").text if book.select_one("bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt") != None else "" # 저자 
            publisher = book.select_one("bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt").text if book.select_one("bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt") != None else "" # 출판사
        elif len(book.select("span.bookListItem_define_data__IUMgt")) == 1:
            author = book.select_one("bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt").text if book.select_one("bookListItem_define_item__Jb5MS > span.bookListItem_define_data__IUMgt") != None else "" # 저자 
            publisher = book.select_one("bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt").text if book.select_one("bookListItem_detail_publish__67dDL > span.bookListItem_define_data__IUMgt") != None else ""# 
    else:
        author = ""
        publisher = ""
    if book.select_one("div.bookListItem_detail_date__s7KQe") != None:
        pub_date = book.select_one("div.bookListItem_detail_date__s7KQe").text.strip(". ").replace(".","-") # 출간일
    if book.select("span.bookPrice_price__OagxI > em") != None:
        print("가격부분:",len(book.select("span.bookPrice_price__OagxI > em")))
        if len(book.select("span.bookPrice_price__OagxI > em")) == 2:
            price = book.select("span.bookPrice_price__OagxI > em")[0].text.strip().replace(",","").replace("원","") # 종이책 가격
            eb_price = book.select("span.bookPrice_price__OagxI > em")[1].text.strip().replace(",","").replace("원","") # e북 가격
        elif len(book.select("span.bookPrice_price__OagxI > em")) == 1:
            price = book.select("span.bookPrice_price__OagxI > em")[0].text.replace(",","")# 종이책 가격
            eb_price = 0
    else:
        price = 0
        eb_price = 0
    
    print(f"""
    title: {title},
    author: {author},
    publisher: {publisher},
    pub_date: {pub_date},
    price: {price},
    eb_price: {eb_price}
    """)

가격부분: 0

    title: 핀테크 규제와 실무(2020) (지급결제와 본인증편),
    author: ,
    publisher: ,
    pub_date: 2020-05-04,
    price: 2790,
    eb_price: 0
    
가격부분: 0

    title: 핀테크(Fintech) 3.0 (핀테크 혁명과 금융의 미래),
    author: ,
    publisher: ,
    pub_date: 2015-08-31,
    price: 2790,
    eb_price: 0
    
가격부분: 0

    title: 핀테크(Fintech) 3.0 (핀테크 혁명과 금융의 미래),
    author: ,
    publisher: ,
    pub_date: 2016-09-05,
    price: 2790,
    eb_price: 0
    
가격부분: 1

    title: 핀테크 산업분야 주식투자 가이드,
    author: ,
    publisher: ,
    pub_date: 2021-04-26,
    price: 27000,
    eb_price: 0
    
가격부분: 0

    title: 김변이 알려주는 핀테크의 비밀 (알아야 돈이 보인다),
    author: ,
    publisher: ,
    pub_date: 2019-07-01,
    price: 27000,
    eb_price: 0
    
가격부분: 1

    title: 김변이 알려주는 핀테크의 비밀,
    author: ,
    publisher: ,
    pub_date: 2019-01-24,
    price: 17100,
    eb_price: 0
    
가격부분: 2

    title: 4차 산업혁명시대 핀테크 개인정보보호 (핀테크 기업과 이용자를 위한 개인정보보호 핵심 가이드),
    author: ,
    publisher: ,
    pub_date: 2019-03-20,
    pri

In [70]:
book.select_one("span.bookListItem_text__SL9m9").text

'내일의 핀테크'

In [78]:
len(book.select("span.bookListItem_define_data__IUMgt"))

0

In [64]:
book.select("span.bookPrice_price__OagxI > em")[0].text

'13,500'

In [69]:
book.select("span.bookListItem_define_data__IUMgt")

[]